В этой тетрадке я напишу свои мысли по поводу фильтрации

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import preprocessor as prep
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import numpy as np

from sklearn.preprocessing import MinMaxScaler

Вот так выглядит двумерное распределение дебита от времени измерения. Но время замера протянуто впереёд, полную картину дадут исходные данные

In [ ]:
wells = [252, 658, 1479, 1509]
well = 1509
calculated_data = prep.load_calculated_data_from_csv(f'data/{well}/restore_ml/{well}_restore_1.csv')
input_data = pd.read_csv(f'data/{well}/restore_input_ml/{well}_restore_input.csv',
                         parse_dates=True, index_col = 'Время')
all_data = input_data.join(calculated_data, how = 'outer')
all_data.to_csv(f'data/{well}/restore_ml/{well}_calc_and_input.csv')

plt.scatter(all_data['Объемный дебит жидкости (СУ)'], all_data['Время замера фактическое (СУ)'])
plt.xlabel('Дебит')
plt.ylabel('Время замера')
plt.title(f'Well {well}')
plt.show()
print(len(all_data['Объемный дебит жидкости (СУ)']))

In [ ]:
input_data.columns.values

In [ ]:
pfig = plt.figure(figsize=(16,5))
plt.scatter(all_data['Объемный дебит жидкости (СУ)'].index,
            all_data['Объемный дебит жидкости (СУ)'].values.reshape((-1, 1)), c='red',
            label='not filtered_data')
plt.xlabel('Date')
plt.ylabel('Q_liq')
plt.title(f'Well {well}')
plt.show()

А вот так выглядят настоящие данные

In [ ]:
cs_d = pd.read_csv(f'data/{well}/init_edit/{well}_first_edit.csv')

In [ ]:
type(cs_d.dropna(subset=['Объемный дебит жидкости'])['Время'])

In [ ]:


sc = MinMaxScaler()
# строим дебит
fig = plt.figure(figsize=(16,10))
ax =plt.subplot(2, 1, 1)
ax.plot(pd.to_datetime(cs_d.dropna(subset=['Объемный дебит жидкости'])['Время']),
         sc.fit_transform(cs_d['Объемный дебит жидкости'].dropna().values.reshape((-1, 1))),
           label='Q_liq')

sc_data  = cs_d[['Время замера фактическое', 'Время']].dropna().join(cs_d['Объемный дебит жидкости'].dropna().to_frame(),
                                                                     how='inner')
# строим флажки там где известно время замера
ax.scatter(pd.to_datetime(sc_data.dropna(subset=['Объемный дебит жидкости'])['Время']),
           sc.transform(sc_data['Объемный дебит жидкости'].dropna().values.reshape((-1, 1))), c='red',
           label='known measuring time')

ax.set(xlabel='Date: day/month', ylabel='scaled to (0;1) values',
       title=f'Q_liq and true measurs of measuring time, well={well}, first_edit'
      )
plt.legend(loc='upper right')
ax.xaxis.set_major_formatter(DateFormatter('%d/%m'))
plt.show()


fig = plt.figure(figsize=(16,10))           
ax =plt.subplot(2, 1, 2)
ax.plot(pd.to_datetime(sc_data.dropna(subset=['Объемный дебит жидкости'])['Время'].values),
            sc_data['Время замера фактическое'].dropna().values.reshape((-1, 1)), c='green',
            label='known measuring time')
ax.set(xlabel='Date: day/month', ylabel='seconds',
           title=f'measuring_time, well={well}')
plt.legend(loc='upper right')
ax.xaxis.set_major_formatter(DateFormatter('%d/%m'))
plt.show()

known_part = sc_data["Объемный дебит жидкости"].dropna().shape[0]/cs_d["Объемный дебит жидкости"].dropna().shape[0]
print(f'Известно замеров: {known_part*100}%')

Видно, что замеры в области пика не производились, это наводит на мысль, что делать выводы о некорректности замеров только лишь по времени замера - некорректно, из-за отсутствия адекватного набора данных

Будем определять аномалии как отклонение от среднего

In [ ]:
m = all_data['Объемный дебит жидкости (СУ)'].mean()
sigma =  all_data['Объемный дебит жидкости (СУ)'].values.std()

Ещё мы наверняка знаем, что бывают плохие замеры. Убирать их нужно, основываясь на исходных данных, не протянутых

Данные с ЦС на этапе `input` от ресэмпленны, сделаем это тоже

In [ ]:
all_data_f = all_data.copy()
all_data_f = all_data_f[all_data_f['Объемный дебит жидкости (СУ)'] <= m + 3 * sigma]
all_data_f = all_data_f[all_data_f['Объемный дебит жидкости (СУ)'] >= m - 2 * sigma]

res_cs = cs_d[['Время замера фактическое', 'Время']].dropna().set_index('Время')
res_cs.index = pd.to_datetime(res_cs.index)
res_cs = res_cs.resample('3h').mean()

all_data_f = all_data_f.join(res_cs)
all_data_f['Время замера фактическое'].fillna(np.inf, inplace=True)
all_data_f = all_data_f[all_data_f['Время замера фактическое'].dropna() - all_data_f['Время замера плановое (СУ)'] - 30 > 0]

Построим результаты фильтрации

In [ ]:
pfig = plt.figure(figsize=(16,5))
plt.scatter(all_data['Объемный дебит жидкости (СУ)'].dropna().index,
            all_data['Объемный дебит жидкости (СУ)'].dropna().values.reshape((-1, 1)), c='red',
            label='not filtered_data')
plt.plot(all_data_f['Объемный дебит жидкости (СУ)'].dropna().index,
            all_data_f['Объемный дебит жидкости (СУ)'].dropna().values.reshape((-1, 1)), '-o', c='green',
            label='filtered_data')
plt.title(f'Q_liq, well={well}')
plt.ylabel('scaled to (0,1) values')
plt.xlabel('index')
plt.legend(loc='upper right')
plt.show()

нормал, теперь потестим функции, которые я реализовал
# Описание модуля Filtration

In [ ]:
from Filtration import Droppers

In [ ]:
input_data = pd.read_csv(f'data/{well}/restore_input_ml/{well}_restore_input.csv',
                         parse_dates=True, index_col = 'Время')

In [ ]:
input_sig = Droppers.get_filtred_by_sigma(input_data)

In [ ]:
pfig = plt.figure(figsize=(16,5))
plt.scatter(input_data.index, input_data['Объемный дебит жидкости (СУ)'], c='red',
            label='init data')

plt.plot(input_sig.index, input_sig['Объемный дебит жидкости (СУ)'], '-o', c='green',
            label='filtered_data with stats')
plt.title(f'Q_liq, well={well}')
plt.ylabel('scaled to (0,1) values')
plt.xlabel('index')
plt.legend(loc='upper right')
plt.show()

Видно, что что-то отбросилось

In [ ]:
first_edit_data = pd.read_csv(f'data/{well}/init_edit/{well}_first_edit.csv')

In [ ]:
correct_input = Droppers.get_filtred_by_measurng_time(input_sig, first_edit_data)

In [ ]:
pfig = plt.figure(figsize=(16,5))
plt.scatter(input_sig.index, input_sig['Объемный дебит жидкости (СУ)'], c='red',
            label='filtered with stats')

plt.plot(correct_input.index, correct_input['Объемный дебит жидкости (СУ)'], '-o', c='green',
            label='filtered with stats and measuring time')
plt.title(f'Q_liq, well={well}')
plt.ylabel('scaled to (0,1) values')
plt.xlabel('index')
plt.legend(loc='upper right')
plt.show()